# Hit prediction

In [1]:
# Import libraries
import pandas as pd
import os
import re
import billboard
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from langdetect import detect
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
pd.options.mode.chained_assignment = None

In [2]:
# Spotify authorization
spotify_client_id = os.environ.get('spotify_client_id')
spotify_client_secret = os.environ.get('spotify_client_secret')
client_credentials_manager = SpotifyClientCredentials(client_id = spotify_client_id, client_secret = spotify_client_secret)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

In [3]:
# Download data from last week
newest_chart = billboard.ChartData('hot-100')

In [4]:
# Creating Data Frame for data from last week
song_titles, artists, positions = [[] for i in range(3)]

position = 0
for i in newest_chart:
    position += 1
    positions.append(position)
    song = str(i).split(' by ')
    song_titles.append(song[0].strip("\'"))
    artists.append(song[1])

In [5]:
billboard_weekly_wrap_up = pd.DataFrame({'Position': positions, 'Artist': artists, 'Song Title': song_titles})
display(billboard_weekly_wrap_up)

,Position,Artist,Song Title
0,1,Harry Styles,As It Was
1,2,Jack Harlow,First Class
2,3,Future Featuring Drake & Tems,Wait For U
3,4,Lizzo,About Damn Time
4,5,Glass Animals,Heat Waves
...,...,...,...
95,96,Future,Love You Better
96,97,Keith Urban,Wild Hearts
97,98,Jon Pardi,Last Night Lonely
98,99,Tim McGraw,7500 OBO


In [6]:
# Spliting authors by 'and', 'featuring', 'feat.', Featuring', 'Feat.', '&' to make api searching easier and more efficient
artists = billboard_weekly_wrap_up['Artist'].to_list()
split_artists = []
for artist in artists:
    if any(re.findall(r'and|featuring|feat.|Featuring|Feat.|&|X', str(artist))):
        result = re.split(r'and|featuring|feat.|Featuring|Feat.|&|X', str(artist))[0].strip()
        split_artists.append(result)
    else:
        split_artists.append(artist)

billboard_weekly_wrap_up['Split Names'] = split_artists
display(billboard_weekly_wrap_up)

,Position,Artist,Song Title,Split Names
0,1,Harry Styles,As It Was,Harry Styles
1,2,Jack Harlow,First Class,Jack Harlow
2,3,Future Featuring Drake & Tems,Wait For U,Future
3,4,Lizzo,About Damn Time,Lizzo
4,5,Glass Animals,Heat Waves,Glass Animals
...,...,...,...,...
95,96,Future,Love You Better,Future
96,97,Keith Urban,Wild Hearts,Keith Urban
97,98,Jon Pardi,Last Night Lonely,Jon Pardi
98,99,Tim McGraw,7500 OBO,Tim McGraw


In [7]:
# Get Spotify ID
def get_spotify_ID(artist, track):
    sp_id_list = []
    music_genre_list = []
    audio_features = {}
    try:
        track_id = sp.search(q = 'artist:' + artist + ' track:' + track, type = 'track')
        sp_id = track_id['tracks']['items'][0]['id']
    except IndexError:
        sp_id = ''
    return sp_id
        
spotify_ids = []
for i in billboard_weekly_wrap_up.index:
    data = get_spotify_ID(billboard_weekly_wrap_up['Split Names'][i], billboard_weekly_wrap_up['Song Title'][i])
    spotify_ids.append(data)
    
billboard_weekly_wrap_up['Spotify ID'] = spotify_ids

In [8]:
sum(billboard_weekly_wrap_up['Spotify ID'] == '')

2

In [9]:
billboard_weekly_wrap_up = billboard_weekly_wrap_up[billboard_weekly_wrap_up['Spotify ID'] != '']
display(billboard_weekly_wrap_up)

,Position,Artist,Song Title,Split Names,Spotify ID
0,1,Harry Styles,As It Was,Harry Styles,4LRPiXqCikLlN15c3yImP7
1,2,Jack Harlow,First Class,Jack Harlow,1rDQ4oMwGJI7B4tovsBOxc
2,3,Future Featuring Drake & Tems,Wait For U,Future,59nOXPmaKlBfGMDeOVGrIK
3,4,Lizzo,About Damn Time,Lizzo,1PckUlxKqWQs3RlWXVBLw3
4,5,Glass Animals,Heat Waves,Glass Animals,3USxtqRwSYz57Ewm6wWRMp
...,...,...,...,...,...
95,96,Future,Love You Better,Future,4XJRpBOG0bU3Nxnvam3FnC
96,97,Keith Urban,Wild Hearts,Keith Urban,0BQpyaFrb78b9uobWNheqx
97,98,Jon Pardi,Last Night Lonely,Jon Pardi,3SeeaIXnrCfT88vHVCRWHl
98,99,Tim McGraw,7500 OBO,Tim McGraw,7A4WmJcWx6KrVRfQ2CP8e7


In [10]:
# Function returns audio features for specific Spotify ID
def get_audio_features(spotify_id):
    af = pd.DataFrame(sp.audio_features(spotify_id))
    return af

temp_audio_features_list = []
for i in billboard_weekly_wrap_up['Spotify ID']:
    temp_song_audio_features = get_audio_features(i)
    temp_audio_features_list.append(temp_song_audio_features)
temp_audio_features_df = pd.concat(temp_audio_features_list)
billboard_weekly_wrap_up = billboard_weekly_wrap_up.merge(temp_audio_features_df, how = 'inner', left_on = 'Spotify ID', right_on = 'id')
display(billboard_weekly_wrap_up)

,Position,Artist,Song Title,Split Names,Spotify ID,danceability,energy,key,loudness,mode,...,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,1,Harry Styles,As It Was,Harry Styles,4LRPiXqCikLlN15c3yImP7,0.520,0.731,6,-5.338,0,...,0.3110,0.662,173.930,audio_features,4LRPiXqCikLlN15c3yImP7,spotify:track:4LRPiXqCikLlN15c3yImP7,https://api.spotify.com/v1/tracks/4LRPiXqCikLl...,https://api.spotify.com/v1/audio-analysis/4LRP...,167303,4
1,2,Jack Harlow,First Class,Jack Harlow,1rDQ4oMwGJI7B4tovsBOxc,0.905,0.563,8,-6.135,1,...,0.1130,0.324,106.998,audio_features,1rDQ4oMwGJI7B4tovsBOxc,spotify:track:1rDQ4oMwGJI7B4tovsBOxc,https://api.spotify.com/v1/tracks/1rDQ4oMwGJI7...,https://api.spotify.com/v1/audio-analysis/1rDQ...,173948,4
2,3,Future Featuring Drake & Tems,Wait For U,Future,59nOXPmaKlBfGMDeOVGrIK,0.463,0.642,1,-4.474,1,...,0.0686,0.339,83.389,audio_features,59nOXPmaKlBfGMDeOVGrIK,spotify:track:59nOXPmaKlBfGMDeOVGrIK,https://api.spotify.com/v1/tracks/59nOXPmaKlBf...,https://api.spotify.com/v1/audio-analysis/59nO...,189893,4
3,4,Lizzo,About Damn Time,Lizzo,1PckUlxKqWQs3RlWXVBLw3,0.836,0.743,10,-6.305,0,...,0.3350,0.722,108.966,audio_features,1PckUlxKqWQs3RlWXVBLw3,spotify:track:1PckUlxKqWQs3RlWXVBLw3,https://api.spotify.com/v1/tracks/1PckUlxKqWQs...,https://api.spotify.com/v1/audio-analysis/1Pck...,191822,4
4,5,Glass Animals,Heat Waves,Glass Animals,3USxtqRwSYz57Ewm6wWRMp,0.761,0.525,11,-6.900,1,...,0.0921,0.531,80.870,audio_features,3USxtqRwSYz57Ewm6wWRMp,spotify:track:3USxtqRwSYz57Ewm6wWRMp,https://api.spotify.com/v1/tracks/3USxtqRwSYz5...,https://api.spotify.com/v1/audio-analysis/3USx...,238805,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,96,Future,Love You Better,Future,4XJRpBOG0bU3Nxnvam3FnC,0.709,0.357,5,-9.289,1,...,0.1120,0.544,75.494,audio_features,4XJRpBOG0bU3Nxnvam3FnC,spotify:track:4XJRpBOG0bU3Nxnvam3FnC,https://api.spotify.com/v1/tracks/4XJRpBOG0bU3...,https://api.spotify.com/v1/audio-analysis/4XJR...,129480,4
94,97,Keith Urban,Wild Hearts,Keith Urban,0BQpyaFrb78b9uobWNheqx,0.605,0.786,4,-4.138,1,...,0.3470,0.512,116.995,audio_features,0BQpyaFrb78b9uobWNheqx,spotify:track:0BQpyaFrb78b9uobWNheqx,https://api.spotify.com/v1/tracks/0BQpyaFrb78b...,https://api.spotify.com/v1/audio-analysis/0BQp...,182930,4
95,98,Jon Pardi,Last Night Lonely,Jon Pardi,3SeeaIXnrCfT88vHVCRWHl,0.646,0.816,6,-2.650,1,...,0.1790,0.865,102.972,audio_features,3SeeaIXnrCfT88vHVCRWHl,spotify:track:3SeeaIXnrCfT88vHVCRWHl,https://api.spotify.com/v1/tracks/3SeeaIXnrCfT...,https://api.spotify.com/v1/audio-analysis/3See...,181991,4
96,99,Tim McGraw,7500 OBO,Tim McGraw,7A4WmJcWx6KrVRfQ2CP8e7,0.528,0.718,7,-6.954,1,...,0.3040,0.510,155.759,audio_features,7A4WmJcWx6KrVRfQ2CP8e7,spotify:track:7A4WmJcWx6KrVRfQ2CP8e7,https://api.spotify.com/v1/tracks/7A4WmJcWx6Kr...,https://api.spotify.com/v1/audio-analysis/7A4W...,221400,4


In [11]:
# Drop duplicates
billboard_weekly_wrap_up = billboard_weekly_wrap_up.drop_duplicates()
display(billboard_weekly_wrap_up)

,Position,Artist,Song Title,Split Names,Spotify ID,danceability,energy,key,loudness,mode,...,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,1,Harry Styles,As It Was,Harry Styles,4LRPiXqCikLlN15c3yImP7,0.520,0.731,6,-5.338,0,...,0.3110,0.662,173.930,audio_features,4LRPiXqCikLlN15c3yImP7,spotify:track:4LRPiXqCikLlN15c3yImP7,https://api.spotify.com/v1/tracks/4LRPiXqCikLl...,https://api.spotify.com/v1/audio-analysis/4LRP...,167303,4
1,2,Jack Harlow,First Class,Jack Harlow,1rDQ4oMwGJI7B4tovsBOxc,0.905,0.563,8,-6.135,1,...,0.1130,0.324,106.998,audio_features,1rDQ4oMwGJI7B4tovsBOxc,spotify:track:1rDQ4oMwGJI7B4tovsBOxc,https://api.spotify.com/v1/tracks/1rDQ4oMwGJI7...,https://api.spotify.com/v1/audio-analysis/1rDQ...,173948,4
2,3,Future Featuring Drake & Tems,Wait For U,Future,59nOXPmaKlBfGMDeOVGrIK,0.463,0.642,1,-4.474,1,...,0.0686,0.339,83.389,audio_features,59nOXPmaKlBfGMDeOVGrIK,spotify:track:59nOXPmaKlBfGMDeOVGrIK,https://api.spotify.com/v1/tracks/59nOXPmaKlBf...,https://api.spotify.com/v1/audio-analysis/59nO...,189893,4
3,4,Lizzo,About Damn Time,Lizzo,1PckUlxKqWQs3RlWXVBLw3,0.836,0.743,10,-6.305,0,...,0.3350,0.722,108.966,audio_features,1PckUlxKqWQs3RlWXVBLw3,spotify:track:1PckUlxKqWQs3RlWXVBLw3,https://api.spotify.com/v1/tracks/1PckUlxKqWQs...,https://api.spotify.com/v1/audio-analysis/1Pck...,191822,4
4,5,Glass Animals,Heat Waves,Glass Animals,3USxtqRwSYz57Ewm6wWRMp,0.761,0.525,11,-6.900,1,...,0.0921,0.531,80.870,audio_features,3USxtqRwSYz57Ewm6wWRMp,spotify:track:3USxtqRwSYz57Ewm6wWRMp,https://api.spotify.com/v1/tracks/3USxtqRwSYz5...,https://api.spotify.com/v1/audio-analysis/3USx...,238805,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,96,Future,Love You Better,Future,4XJRpBOG0bU3Nxnvam3FnC,0.709,0.357,5,-9.289,1,...,0.1120,0.544,75.494,audio_features,4XJRpBOG0bU3Nxnvam3FnC,spotify:track:4XJRpBOG0bU3Nxnvam3FnC,https://api.spotify.com/v1/tracks/4XJRpBOG0bU3...,https://api.spotify.com/v1/audio-analysis/4XJR...,129480,4
94,97,Keith Urban,Wild Hearts,Keith Urban,0BQpyaFrb78b9uobWNheqx,0.605,0.786,4,-4.138,1,...,0.3470,0.512,116.995,audio_features,0BQpyaFrb78b9uobWNheqx,spotify:track:0BQpyaFrb78b9uobWNheqx,https://api.spotify.com/v1/tracks/0BQpyaFrb78b...,https://api.spotify.com/v1/audio-analysis/0BQp...,182930,4
95,98,Jon Pardi,Last Night Lonely,Jon Pardi,3SeeaIXnrCfT88vHVCRWHl,0.646,0.816,6,-2.650,1,...,0.1790,0.865,102.972,audio_features,3SeeaIXnrCfT88vHVCRWHl,spotify:track:3SeeaIXnrCfT88vHVCRWHl,https://api.spotify.com/v1/tracks/3SeeaIXnrCfT...,https://api.spotify.com/v1/audio-analysis/3See...,181991,4
96,99,Tim McGraw,7500 OBO,Tim McGraw,7A4WmJcWx6KrVRfQ2CP8e7,0.528,0.718,7,-6.954,1,...,0.3040,0.510,155.759,audio_features,7A4WmJcWx6KrVRfQ2CP8e7,spotify:track:7A4WmJcWx6KrVRfQ2CP8e7,https://api.spotify.com/v1/tracks/7A4WmJcWx6Kr...,https://api.spotify.com/v1/audio-analysis/7A4W...,221400,4


In [12]:
# Load data
all_songs = pd.read_csv('spotify_data.csv')
songs = all_songs[all_songs['Year'] > 2004]

In [13]:
songs['Is hit'] = [1 if i <= 10 else 0 for i in songs['Position']]

In [14]:
# Count hits and non-hits
songs['Is hit'].value_counts()

0    1131
1     138
Name: Is hit, dtype: int64

In [15]:
# Split data into train and test subsets
X = songs[['acousticness', 'danceability', 'duration_ms', 'energy', 'instrumentalness', 'key', 'liveness', 'loudness', 'mode', 'speechiness', 'tempo', 'time_signature', 'valence']]
y = songs['Is hit']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 55)

print('Train data shape: {}'.format(X_train.shape))
print('Test data shape: {}'.format(X_test.shape))

Train data shape: (951, 13)
Test data shape: (318, 13)


In [16]:
# Handling imbalanced train data with smote
oversample = SMOTE()
X_smote, y_smote = oversample.fit_resample(X_train, y_train)

In [17]:
print(X_smote.shape, y_smote.shape)

(1684, 13) (1684,)


In [18]:
# Random forest model
random_forest = RandomForestClassifier(n_estimators = 2000, random_state = 3)
random_forest.fit(X_smote, y_smote)
y_pred = random_forest.predict(X_test)

In [19]:
# Count predicted values
pd.Series(y_pred).value_counts()

0    291
1     27
dtype: int64

### Evaluation

In [20]:
print("Accuracy score: ", accuracy_score(y_test, y_pred))

Accuracy score:  0.8364779874213837


In [21]:
print("Recall score: ", recall_score(y_test, y_pred))

Recall score:  0.06896551724137931


In [22]:
print("Precision score: ", precision_score(y_test, y_pred))

Precision score:  0.07407407407407407


In [23]:
print("F1 score: ", f1_score(y_test, y_pred))

F1 score:  0.07142857142857142


In [24]:
# Data from the last week
display(billboard_weekly_wrap_up)

,Position,Artist,Song Title,Split Names,Spotify ID,danceability,energy,key,loudness,mode,...,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,1,Harry Styles,As It Was,Harry Styles,4LRPiXqCikLlN15c3yImP7,0.520,0.731,6,-5.338,0,...,0.3110,0.662,173.930,audio_features,4LRPiXqCikLlN15c3yImP7,spotify:track:4LRPiXqCikLlN15c3yImP7,https://api.spotify.com/v1/tracks/4LRPiXqCikLl...,https://api.spotify.com/v1/audio-analysis/4LRP...,167303,4
1,2,Jack Harlow,First Class,Jack Harlow,1rDQ4oMwGJI7B4tovsBOxc,0.905,0.563,8,-6.135,1,...,0.1130,0.324,106.998,audio_features,1rDQ4oMwGJI7B4tovsBOxc,spotify:track:1rDQ4oMwGJI7B4tovsBOxc,https://api.spotify.com/v1/tracks/1rDQ4oMwGJI7...,https://api.spotify.com/v1/audio-analysis/1rDQ...,173948,4
2,3,Future Featuring Drake & Tems,Wait For U,Future,59nOXPmaKlBfGMDeOVGrIK,0.463,0.642,1,-4.474,1,...,0.0686,0.339,83.389,audio_features,59nOXPmaKlBfGMDeOVGrIK,spotify:track:59nOXPmaKlBfGMDeOVGrIK,https://api.spotify.com/v1/tracks/59nOXPmaKlBf...,https://api.spotify.com/v1/audio-analysis/59nO...,189893,4
3,4,Lizzo,About Damn Time,Lizzo,1PckUlxKqWQs3RlWXVBLw3,0.836,0.743,10,-6.305,0,...,0.3350,0.722,108.966,audio_features,1PckUlxKqWQs3RlWXVBLw3,spotify:track:1PckUlxKqWQs3RlWXVBLw3,https://api.spotify.com/v1/tracks/1PckUlxKqWQs...,https://api.spotify.com/v1/audio-analysis/1Pck...,191822,4
4,5,Glass Animals,Heat Waves,Glass Animals,3USxtqRwSYz57Ewm6wWRMp,0.761,0.525,11,-6.900,1,...,0.0921,0.531,80.870,audio_features,3USxtqRwSYz57Ewm6wWRMp,spotify:track:3USxtqRwSYz57Ewm6wWRMp,https://api.spotify.com/v1/tracks/3USxtqRwSYz5...,https://api.spotify.com/v1/audio-analysis/3USx...,238805,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,96,Future,Love You Better,Future,4XJRpBOG0bU3Nxnvam3FnC,0.709,0.357,5,-9.289,1,...,0.1120,0.544,75.494,audio_features,4XJRpBOG0bU3Nxnvam3FnC,spotify:track:4XJRpBOG0bU3Nxnvam3FnC,https://api.spotify.com/v1/tracks/4XJRpBOG0bU3...,https://api.spotify.com/v1/audio-analysis/4XJR...,129480,4
94,97,Keith Urban,Wild Hearts,Keith Urban,0BQpyaFrb78b9uobWNheqx,0.605,0.786,4,-4.138,1,...,0.3470,0.512,116.995,audio_features,0BQpyaFrb78b9uobWNheqx,spotify:track:0BQpyaFrb78b9uobWNheqx,https://api.spotify.com/v1/tracks/0BQpyaFrb78b...,https://api.spotify.com/v1/audio-analysis/0BQp...,182930,4
95,98,Jon Pardi,Last Night Lonely,Jon Pardi,3SeeaIXnrCfT88vHVCRWHl,0.646,0.816,6,-2.650,1,...,0.1790,0.865,102.972,audio_features,3SeeaIXnrCfT88vHVCRWHl,spotify:track:3SeeaIXnrCfT88vHVCRWHl,https://api.spotify.com/v1/tracks/3SeeaIXnrCfT...,https://api.spotify.com/v1/audio-analysis/3See...,181991,4
96,99,Tim McGraw,7500 OBO,Tim McGraw,7A4WmJcWx6KrVRfQ2CP8e7,0.528,0.718,7,-6.954,1,...,0.3040,0.510,155.759,audio_features,7A4WmJcWx6KrVRfQ2CP8e7,spotify:track:7A4WmJcWx6KrVRfQ2CP8e7,https://api.spotify.com/v1/tracks/7A4WmJcWx6Kr...,https://api.spotify.com/v1/audio-analysis/7A4W...,221400,4


In [25]:
# Delete songs from last week which appeared in year-end chart in 2021
cond = billboard_weekly_wrap_up['Spotify ID'].isin(songs['Spotify ID'])
billboard_weekly_wrap_up.drop(billboard_weekly_wrap_up[cond].index, inplace = True)
display(billboard_weekly_wrap_up)

,Position,Artist,Song Title,Split Names,Spotify ID,danceability,energy,key,loudness,mode,...,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,1,Harry Styles,As It Was,Harry Styles,4LRPiXqCikLlN15c3yImP7,0.520,0.731,6,-5.338,0,...,0.3110,0.662,173.930,audio_features,4LRPiXqCikLlN15c3yImP7,spotify:track:4LRPiXqCikLlN15c3yImP7,https://api.spotify.com/v1/tracks/4LRPiXqCikLl...,https://api.spotify.com/v1/audio-analysis/4LRP...,167303,4
1,2,Jack Harlow,First Class,Jack Harlow,1rDQ4oMwGJI7B4tovsBOxc,0.905,0.563,8,-6.135,1,...,0.1130,0.324,106.998,audio_features,1rDQ4oMwGJI7B4tovsBOxc,spotify:track:1rDQ4oMwGJI7B4tovsBOxc,https://api.spotify.com/v1/tracks/1rDQ4oMwGJI7...,https://api.spotify.com/v1/audio-analysis/1rDQ...,173948,4
2,3,Future Featuring Drake & Tems,Wait For U,Future,59nOXPmaKlBfGMDeOVGrIK,0.463,0.642,1,-4.474,1,...,0.0686,0.339,83.389,audio_features,59nOXPmaKlBfGMDeOVGrIK,spotify:track:59nOXPmaKlBfGMDeOVGrIK,https://api.spotify.com/v1/tracks/59nOXPmaKlBf...,https://api.spotify.com/v1/audio-analysis/59nO...,189893,4
3,4,Lizzo,About Damn Time,Lizzo,1PckUlxKqWQs3RlWXVBLw3,0.836,0.743,10,-6.305,0,...,0.3350,0.722,108.966,audio_features,1PckUlxKqWQs3RlWXVBLw3,spotify:track:1PckUlxKqWQs3RlWXVBLw3,https://api.spotify.com/v1/tracks/1PckUlxKqWQs...,https://api.spotify.com/v1/audio-analysis/1Pck...,191822,4
5,6,Latto,Big Energy,Latto,6Zu3aw7FfjAF9WA0fA81Oq,0.935,0.807,11,-3.838,0,...,0.3490,0.813,106.017,audio_features,6Zu3aw7FfjAF9WA0fA81Oq,spotify:track:6Zu3aw7FfjAF9WA0fA81Oq,https://api.spotify.com/v1/tracks/6Zu3aw7FfjAF...,https://api.spotify.com/v1/audio-analysis/6Zu3...,173182,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,96,Future,Love You Better,Future,4XJRpBOG0bU3Nxnvam3FnC,0.709,0.357,5,-9.289,1,...,0.1120,0.544,75.494,audio_features,4XJRpBOG0bU3Nxnvam3FnC,spotify:track:4XJRpBOG0bU3Nxnvam3FnC,https://api.spotify.com/v1/tracks/4XJRpBOG0bU3...,https://api.spotify.com/v1/audio-analysis/4XJR...,129480,4
94,97,Keith Urban,Wild Hearts,Keith Urban,0BQpyaFrb78b9uobWNheqx,0.605,0.786,4,-4.138,1,...,0.3470,0.512,116.995,audio_features,0BQpyaFrb78b9uobWNheqx,spotify:track:0BQpyaFrb78b9uobWNheqx,https://api.spotify.com/v1/tracks/0BQpyaFrb78b...,https://api.spotify.com/v1/audio-analysis/0BQp...,182930,4
95,98,Jon Pardi,Last Night Lonely,Jon Pardi,3SeeaIXnrCfT88vHVCRWHl,0.646,0.816,6,-2.650,1,...,0.1790,0.865,102.972,audio_features,3SeeaIXnrCfT88vHVCRWHl,spotify:track:3SeeaIXnrCfT88vHVCRWHl,https://api.spotify.com/v1/tracks/3SeeaIXnrCfT...,https://api.spotify.com/v1/audio-analysis/3See...,181991,4
96,99,Tim McGraw,7500 OBO,Tim McGraw,7A4WmJcWx6KrVRfQ2CP8e7,0.528,0.718,7,-6.954,1,...,0.3040,0.510,155.759,audio_features,7A4WmJcWx6KrVRfQ2CP8e7,spotify:track:7A4WmJcWx6KrVRfQ2CP8e7,https://api.spotify.com/v1/tracks/7A4WmJcWx6Kr...,https://api.spotify.com/v1/audio-analysis/7A4W...,221400,4


In [26]:
# Results
chart_audio_features = billboard_weekly_wrap_up[['acousticness', 'danceability', 'duration_ms', 'energy', 'instrumentalness', 'key', 'liveness', 'loudness', 'mode', 'speechiness', 'tempo', 'time_signature', 'valence']]
billboard_weekly_wrap_up['Predict as hit'] = random_forest.predict(chart_audio_features)
display(billboard_weekly_wrap_up[billboard_weekly_wrap_up['Predict as hit'] == 1])

,Position,Artist,Song Title,Split Names,Spotify ID,danceability,energy,key,loudness,mode,...,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,Predict as hit
3,4,Lizzo,About Damn Time,Lizzo,1PckUlxKqWQs3RlWXVBLw3,0.836,0.743,10,-6.305,0,...,0.722,108.966,audio_features,1PckUlxKqWQs3RlWXVBLw3,spotify:track:1PckUlxKqWQs3RlWXVBLw3,https://api.spotify.com/v1/tracks/1PckUlxKqWQs...,https://api.spotify.com/v1/audio-analysis/1Pck...,191822,4,1
6,7,Bad Bunny & Chencho Corleone,Me Porto Bonito,Bad Bunny,6Sq7ltF9Qa7SNFBsV5Cogx,0.911,0.712,1,-5.105,0,...,0.425,92.005,audio_features,6Sq7ltF9Qa7SNFBsV5Cogx,spotify:track:6Sq7ltF9Qa7SNFBsV5Cogx,https://api.spotify.com/v1/tracks/6Sq7ltF9Qa7S...,https://api.spotify.com/v1/audio-analysis/6Sq7...,178567,4,1
10,11,Bad Bunny,Titi Me Pregunto,Bad Bunny,1IHWl5LamUGEuP4ozKQSXZ,0.650,0.715,5,-5.198,0,...,0.187,106.672,audio_features,1IHWl5LamUGEuP4ozKQSXZ,spotify:track:1IHWl5LamUGEuP4ozKQSXZ,https://api.spotify.com/v1/tracks/1IHWl5LamUGE...,https://api.spotify.com/v1/audio-analysis/1IHW...,243717,4,1
28,29,Elton John & Dua Lipa,Cold Heart (PNAU Remix),Elton John,6zSpb8dQRaw0M1dK8PBwQz,0.796,0.798,1,-6.312,1,...,0.942,116.032,audio_features,6zSpb8dQRaw0M1dK8PBwQz,spotify:track:6zSpb8dQRaw0M1dK8PBwQz,https://api.spotify.com/v1/tracks/6zSpb8dQRaw0...,https://api.spotify.com/v1/audio-analysis/6zSp...,202735,4,1
33,34,Karol G,Provenza,Karol G,7dSZ6zGTQx66c2GF91xCrb,0.870,0.516,1,-8.006,1,...,0.530,111.005,audio_features,7dSZ6zGTQx66c2GF91xCrb,spotify:track:7dSZ6zGTQx66c2GF91xCrb,https://api.spotify.com/v1/tracks/7dSZ6zGTQx66...,https://api.spotify.com/v1/audio-analysis/7dSZ...,210200,4,1
40,41,Harry Styles,Music For A Sushi Restaurant,Harry Styles,5LYMamLv12UPbemOaTPyeV,0.720,0.715,11,-4.652,1,...,0.357,107.041,audio_features,5LYMamLv12UPbemOaTPyeV,spotify:track:5LYMamLv12UPbemOaTPyeV,https://api.spotify.com/v1/tracks/5LYMamLv12UP...,https://api.spotify.com/v1/audio-analysis/5LYM...,193813,4,1
78,80,Russell Dickerson & Jake Scott,She Likes It,Russell Dickerson,6VVqPBFZIbcmv14zNfOc13,0.790,0.325,4,-11.689,1,...,0.539,126.877,audio_features,6VVqPBFZIbcmv14zNfOc13,spotify:track:6VVqPBFZIbcmv14zNfOc13,https://api.spotify.com/v1/tracks/6VVqPBFZIbcm...,https://api.spotify.com/v1/audio-analysis/6VVq...,163311,4,1
